# Pair Programming ETL Transformación II

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I.

In [39]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta

In [34]:
df=pd.read_csv("./df_unido.csv",index_col=0) #  union ataques y clima
df.reset_index(inplace=True)
df.head()


,index,index_x,year,type,country,age,species_,fecha_limpia,fatal,sex,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_y,longitud_y
0,0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
1,1,1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
2,2,2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
3,3,7,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
4,4,8,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882


In [41]:
# Cargamos el primer archivo
df_ataques = pd.read_csv("./attacks_limpieza_completa.csv", index_col = 0)
df_ataques.reset_index(inplace=True)
df_ataques.head()

,index,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,...,1,0,0,1,0,0,4,7,1,9.116327
1,1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,...,1,0,0,1,0,0,3,7,7,3.804860
2,2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,...,1,0,0,1,0,0,3,7,3,8.403629
3,3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,...,1,0,0,1,0,0,3,7,7,6.313925
4,4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.658512,...,1,0,0,1,0,0,2,7,4,6.313925


In [35]:
pais= "USA","Australia","South Africa","New Zealand","Papua New Guinea"
latitud_longitud= [39.7837304,-100.445882], [-24.7761086,134.755],[-28.8166236,24.991639], [-41.5000831,172.8344077], [-5.6816069,144.2489081]

diccionario = dict(zip(pais, latitud_longitud))
diccionario

{'USA': [39.7837304, -100.445882],
 'Australia': [-24.7761086, 134.755],
 'South Africa': [-28.8166236, 24.991639],
 'New Zealand': [-41.5000831, 172.8344077],
 'Papua New Guinea': [-5.6816069, 144.2489081]}

In [54]:
class Extraccion: #self una variable global para toda nuestra clas, en init (si tengo dic,otro), self.dic


    def __init__(self, diccionario):
        self.diccionario =diccionario


    def llamar_API(self,producto):
        self.producto = producto
        df_completo= pd.DataFrame()

        for key, value in diccionario.items():

            lon= value[1]
            lat= value[0]
            pais=key
            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json' 
            response = requests.get(url=url)
            codigo_estado= response.status_code #200
            razon_estado = response.reason 

            df_API = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))

            df_API["pais"]=pais
            df_API["latitud"]=lat
            df_API["longitud"]=lon 

            hoy = datetime.now()
            hoy = datetime.strftime(hoy, '%Y-%m-%d') # va a estar en formato YYYY-MM-DD
            df_API["fecha"] = hoy 

            df_completo = pd.concat([df_completo,df_API], axis = 0)  #mete aqui todo lo que te llegue de df_API
    
        return df_completo


    def chequear_datos(self,df_completo): 
    
        print("Las columnas son:", "\n")
        print(list((df_completo.columns)))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_completo.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_completo.isnull().sum() / df_completo.shape[0]) *  100)

    def limpiar_dataframe(self,df_completo):
        df_completo["rh_profile"] =df_completo["rh_profile"].apply(ast.literal_eval) #castear
        rh= df_completo["rh_profile"].apply(pd.Series) 

        for i in range(len(rh.columns)):
            nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
            valores = list(rh[i].apply(pd.Series)["rh"])#tenemos que convertirlo a lista para q funcione el insert
            df_completo.insert(i, nombre, valores)#insert acepta listas


        df_completo['wind_profile']= df_completo['wind_profile'].apply(ast.literal_eval)
        direction= df_completo["wind_profile"].apply(pd.Series)
        for i in range(len(direction.columns)):
            nombre = "wind_" + str(direction[i].apply(pd.Series)["layer"][0]) 
            valores = list(direction[i].apply(pd.Series)["direction"])#tenemos que convertirlo a lista para q funcione el insert
            df_completo.insert(i, nombre, valores)#insert acepta listas
        

        speed= df_completo["wind_profile"].apply(pd.Series)
        for i in range(len(speed.columns)):
            nombre = "wind_speed_" + str(speed[i].apply(pd.Series)["layer"][0]) 
            valores = list(speed[i].apply(pd.Series)["speed"])#tenemos que convertirlo a lista para q funcione el insert
            df_completo.insert(i, nombre, valores)#insert acepta listas


        df_completo= df_completo.groupby("pais").mean()
        df_completo= pd.DataFrame(df_completo.reset_index())
        df_completo["pais"]= df_completo["pais"].lower()

        return df_completo

    def juntar(df_ataques,df_completo):

        df_unido = pd.merge(df_ataques,df_completo, left_on=["country"], right_on=["pais"])

        return df_unido
    
    def unir_anterior(df_unido,df):

        df_final = pd.merge(df_ataques,df, on=["country"])

        #Guardamos los archivos
        df_final.to_pickle('datos/datos_actualizados.pkl')
        df_final.to_csv('datos/datos_actualizados.csv')

        return df_final

    

In [55]:
api = Extraccion(diccionario)

Llamamos al producto meteo

In [56]:
df_API = api.llamar_API("meteo")
df_API.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,fecha
0,3,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 120, 'speed':...",23,2,4,1022,none,0,0,95,2,USA,39.78373,-100.445882,2022-05-19
1,6,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 130, 'speed':...",23,-1,5,1022,none,0,0,110,2,USA,39.78373,-100.445882,2022-05-19


Chequear los datos

In [ ]:
api.chequear_datos(diccionario)

Limpiamos los dataframes

In [ ]:
api.limpiar_api(diccionario)


Juntamos los dataframes

In [ ]:
api.juntar_df(diccionario) 


In [ ]:
api.unir_anterior(diccionario)
